In [3]:
import pandas as pd
import numpy as np 
from glob import glob
import cv2
from skimage import io 
from tqdm import tqdm
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer
from tensorflow.keras import layers
import tensorflow.keras.optimizers as Optimizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Flatten, Activation, 
    MaxPooling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D, GlobalMaxPooling2D, add
)
import pandas as pd
import numpy as np 
from glob import glob
import cv2
from skimage import io 
from tqdm import tqdm
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer
from tensorflow.keras import layers
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.applications.inception_v3 import preprocess_input

# ResNet Verison 5


In [7]:
CFG2 = dict(
        batch_size        =  32,     # 8; 16; 32; 64; bigger batch size => moemry allocation issue
        epochs            =  100,   # 5; 10; 20;
        verbose           =   1,    # 0; 1
        workers           =   4,    # 1; 2; 3
        optimizer         = 'adam', # 'SGD', 'RMSprop'

        RANDOM_STATE      =  123,   
    
        # Path to save a model
        path_model        = '../working/',

        # Images sizes
        img_size          = 224, 
        img_height        = 224, 
        img_width         = 224, 

        # Images augs
        ROTATION          = 180.0,
        ZOOM              =  10.0,
        ZOOM_RANGE        =  [0.9,1.1],
        HZOOM             =  8.0, #maybe try 8.0?
        WZOOM             =  8.0, #maybe try 8.0?
        HSHIFT            =  8.0, #maybe try 8.0?
        WSHIFT            =  8.0, #maybe try 8.0?
        SHEAR             =   2.0, #maybe try 2.0?
        HFLIP             = True,
        VFLIP             = True,

        # Postprocessing
        label_smooth_fac  =  0.05,  # 0.01; 0.05; 0.1; 0.2;     Label smoothing is used when the loss function is cross entropy, and the model applies the softmax function to the penultimate layer’s logit vectors z to compute its output probabilities p. 
)

#CHANGED ZOOM, SHift, smooth_fac and Batchsize

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3,
    rotation_range            = CFG2['ROTATION'],
    zoom_range                = CFG2['ZOOM_RANGE'],  #is for randomly zooming inside pictures  
    horizontal_flip           = CFG2['HFLIP'], #is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).
    vertical_flip             = CFG2['VFLIP'],
    height_shift_range        = CFG2['HSHIFT'],
    width_shift_range         = CFG2['WSHIFT'],
    shear_range               = CFG2['SHEAR'], #randomly applying shearing transformations
    channel_shift_range       = 0.0,
    brightness_range          = None,
    fill_mode                 = 'nearest',      #is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift                     
    )

valid_generator = ImageDataGenerator(rescale=1./255, validation_split=0.3)              # no aug for valid
test_generator  = ImageDataGenerator(rescale=1./255)                                    # no aug for test
# Train data
train_generator = train_datagen.flow_from_directory("Image_dir_2",
                                                    subset='training',                  # to read train/valid from same directory 
                                                    target_size=(CFG2['img_size'], CFG2['img_size']),
                                                    batch_size = CFG2['batch_size'],
                                                    class_mode='categorical',
                                                    )

# Validation data
valid_generator = valid_generator.flow_from_directory("Image_dir_2",
                                                     subset='validation',               # to read train/valid from same directory 
                                                     target_size=(CFG2['img_size'], CFG2['img_size']),
                                                     batch_size = CFG2['batch_size'],
                                                     class_mode='categorical'
                                                     ) 
# Test data
test_generator  = test_generator.flow_from_directory("Image_dir_2",
                                                     target_size=(CFG2['img_size'], CFG2['img_size']),
                                                     batch_size = 1,                    # using 1 to easily manage mapping between test_gen & pred
                                                     class_mode='categorical'
                                                     )

Found 17736 images belonging to 8 classes.
Found 7595 images belonging to 8 classes.
Found 25331 images belonging to 8 classes.


In [9]:
model_4_ResNet50 = tf.keras.Sequential([
     tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        weights='imagenet',
        include_top=False
    ),
    
    GlobalAveragePooling2D(),
    
    Dense(1024, activation = 'relu'), 
    Dropout(0.5), 
    BatchNormalization(),
    
    Dense(256, activation='relu'), 
    Dropout(0.3), 
    BatchNormalization(),
    
    Dense(64, activation='relu'), 
    Dropout(0.2), 
    BatchNormalization(),
    
    Dense(8, activation='softmax') # num classes = 8
    
])
    
model_4_ResNet50.compile(
    optimizer = CFG2['optimizer'],
    loss = "categorical_crossentropy",
    metrics=['accuracy']
)

In [10]:
#tf.function-decorated function tried to create variables on non-first call'. 
tf.config.run_functions_eagerly(True) # otherwise error

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 10)
cb_checkpointer  = ModelCheckpoint(filepath = CFG['path_model']+'ResNet50_3-{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor  = 'val_loss', 
                                   verbose  = CFG['verbose'], 
                                   save_best_only=True, 
                                   mode='min'
                                  )

callbacks_list = [cb_checkpointer, cb_early_stopper]

history3 = model_4_ResNet50.fit(train_generator, 
                             epochs=55, 
                             workers=CFG2['workers'],
                             validation_data=valid_generator, 
                            )

/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/55
555/555 [==============================] - 27486s 50s/step - loss: 1.4535 - accuracy: 0.5297 - val_loss: 1.9752 - val_accuracy: 0.1311
Epoch 2/55
555/555 [==============================] - 21324s 38s/step - loss: 1.1039 - accuracy: 0.6189 - val_loss: 1.3347 - val_accuracy: 0.5604
Epoch 3/55
555/555 [==============================] - 16024s 29s/step - loss: 1.0618 - accuracy: 0.6321 - val_loss: 1.2459 - val_accuracy: 0.5509
Epoch 4/55
555/555 [==============================] - 12636s 23s/step - loss: 1.0260 - accuracy: 0.6427 - val_loss: 1.4752 - val_accuracy: 0.5322
Epoch 5/55
555/555 [==============================] - 16897s 30s/step - loss: 1.0037 - accuracy: 0.6486 - val_loss: 1.5948 - val_accuracy: 0.5360
Epoch 6/55
555/555 [==============================] - 16133s 29s/step - loss: 0.9843 - accuracy: 0.6551 - val_loss: 1.3211 - val_accuracy: 0.5577
Epoch 7/55
555/555 [==============================] - 12335s 22s/step - loss: 0.9652 - accuracy: 0.6605 - val_loss: 2.4917 -

KeyboardInterrupt: 

In [12]:
from sklearn.metrics import classification_report,confusion_matrix
predictions1=model_4_ResNet50.predict(valid_generator,steps=len(valid_generator))
y=np.argmax(predictions1,axis=1)

print('Classification Report')
cr=classification_report(y_true=valid_generator.classes,y_pred=y,target_names=valid_generator.class_indices)
print(cr)
evaluates=model_4_ResNet50.evaluate(valid_generator)
print(evaluates)

/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Classification Report
              precision    recall  f1-score   support

   AK_images       0.00      0.00      0.00       260
  BCC_images       0.07      0.01      0.01       996
  BKL_images       0.14      0.02      0.03       787
   DF_images       0.00      0.00      0.00        71
  MEL_images       0.18      0.31      0.22      1356
   NV_images       0.51      0.59      0.55      3862
  SCC_images       0.00      0.00      0.00       188
 VASC_images       0.01      0.04      0.01        75

    accuracy                           0.36      7595
   macro avg       0.11      0.12      0.10      7595
weighted avg       0.31      0.36      0.32      7595



/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/snizhanakurylyuk/opt/anaconda3/envs/metis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

238/238 [==============================] - 1576s 7s/step - loss: 1.4739 - accuracy: 0.4924
[1.4738965034484863, 0.4924292266368866]
